# 01. Tiền xử lý dữ liệu thiết bị PMIS

Notebook này thực hiện các bước tiền xử lý dữ liệu thiết bị máy cắt từ file `devicesPMISMayCat.csv`

## Mục lục
- A. Đọc & phân tích dữ liệu
- B. Làm sạch dữ liệu
  - B.1. Xử lý NULL string
  - B.2. Cắt khoảng trắng thừa
  - B.3. Chuẩn hóa kiểu dữ liệu
  - **B.3.1. Điền DATEMANUFACTURE theo nhóm** *(mới)*
  - B.4. Xử lý dòng trùng lặp
  - **B.5. Loại bỏ CATEGORYID không hợp lệ** *(mới)*
  - **B.6. Loại bỏ dòng thiếu > 50% dữ liệu** *(mới)*
  - **B.7. Loại bỏ ASSETID không hợp lệ** *(mới)*
  - **B.8. Loại bỏ NATIONALFACT bị cấm** *(mới)*
- C. Kiểm tra quy tắc chuẩn hóa
- D. Đánh dấu dữ liệu thiếu
- E. Xuất dữ liệu sạch

In [1]:
# Import thư viện
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Thêm đường dẫn project
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_ROOT)

# Đường dẫn file
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
INPUT_FILE = os.path.join(DATA_DIR, 'devicesPMISMayCat.csv')
OUTPUT_FILE = os.path.join(DATA_DIR, 'devicesPMISMayCat_cleaned.csv')
LOG_DIR = os.path.join(PROJECT_ROOT, 'logs')

# Tạo thư mục logs nếu chưa có
os.makedirs(LOG_DIR, exist_ok=True)

print(f"Project Root: {PROJECT_ROOT}")
print(f"Input File: {INPUT_FILE}")
print(f"Output File: {OUTPUT_FILE}")

Project Root: /home/aispcit/Documents/QuangLV/PMIS v 13
Input File: /home/aispcit/Documents/QuangLV/PMIS v 13/data/devicesPMISMayCat.csv
Output File: /home/aispcit/Documents/QuangLV/PMIS v 13/data/devicesPMISMayCat_cleaned.csv


## A. Đọc & Phân tích dữ liệu

In [2]:
# A.1. Đọc file CSV (delimiter ;, UTF-8)
df = pd.read_csv(INPUT_FILE, delimiter=';', encoding='utf-8')

print("=" * 60)
print("A.1. ĐỌC DỮ LIỆU THÀNH CÔNG")
print("=" * 60)
print(f"Số dòng: {len(df):,}")
print(f"Số cột: {len(df.columns)}")
print(f"\nDanh sách các cột:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

A.1. ĐỌC DỮ LIỆU THÀNH CÔNG
Số dòng: 1,755
Số cột: 32

Danh sách các cột:
   1. CATEGORYID
   2. ASSETID
   3. ASSETDESC
   4. P_MANUFACTURERID
   5. P_MANUFACTURERID_DESC
   6. DATEMANUFACTURE
   7. NATIONALFACT
   8. FIELDDESC
   9. OWNER
  10. OWNER_DESC
  11. LOAI
  12. LOAI_DESC
  13. U_TT
  14. U_TT_DESC
  15. KIEU_DAPHQ
  16. KIEU_DAPHQ_DESC
  17. I_DM
  18. I_DM_DESC
  19. U_DM
  20. U_DM_DESC
  21. KIEU_CD
  22. KIEU_CD_DESC
  23. TG_CATNM
  24. TG_CATNM_DESC
  25. PHA
  26. PHA_DESC
  27. KIEU_MC
  28. KIEU_MC_DESC
  29. KNCDNMDM
  30. KNCDNMDM_DESC
  31. CT_DC
  32. CT_DC_DESC


In [3]:
# A.2. Thống kê số dòng, số cột
print("=" * 60)
print("A.2. THỐNG KÊ CƠ BẢN")
print("=" * 60)
print(f"Tổng số dòng dữ liệu: {len(df):,}")
print(f"Tổng số cột: {len(df.columns)}")
print(f"Bộ nhớ sử dụng: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
print(f"\nThông tin kiểu dữ liệu:")
print(df.dtypes.value_counts())

A.2. THỐNG KÊ CƠ BẢN
Tổng số dòng dữ liệu: 1,755
Tổng số cột: 32
Bộ nhớ sử dụng: 3500.70 KB

Thông tin kiểu dữ liệu:
object     28
float64     4
Name: count, dtype: int64


In [4]:
# A.3. Kiểm tra tỷ lệ NULL theo cột
print("=" * 60)
print("A.3. TỶ LỆ NULL THEO CỘT")
print("=" * 60)

# Kiểm tra giá trị 'NULL' dạng string và NaN
null_stats = []
for col in df.columns:
    nan_count = df[col].isna().sum()
    null_string_count = (df[col] == 'NULL').sum()
    total_missing = nan_count + null_string_count
    null_stats.append({
        'Cột': col,
        'NaN': nan_count,
        'NULL string': null_string_count,
        'Tổng thiếu': total_missing,
        'Tỷ lệ (%)': round(total_missing / len(df) * 100, 2)
    })

null_df = pd.DataFrame(null_stats)
null_df_sorted = null_df.sort_values('Tổng thiếu', ascending=False)
print(null_df_sorted[null_df_sorted['Tổng thiếu'] > 0].to_string(index=False))

A.3. TỶ LỆ NULL THEO CỘT
                  Cột  NaN  NULL string  Tổng thiếu  Tỷ lệ (%)
      DATEMANUFACTURE  160            0         160       9.12
            LOAI_DESC   53            0          53       3.02
      KIEU_DAPHQ_DESC   40            0          40       2.28
            U_TT_DESC   40            0          40       2.28
                 U_TT   40            0          40       2.28
           KIEU_DAPHQ   40            0          40       2.28
                 LOAI   40            0          40       2.28
                  PHA   40            0          40       2.28
        TG_CATNM_DESC   40            0          40       2.28
             TG_CATNM   40            0          40       2.28
         KIEU_CD_DESC   40            0          40       2.28
              KIEU_CD   40            0          40       2.28
            U_DM_DESC   40            0          40       2.28
                 U_DM   40            0          40       2.28
            I_DM_DESC   40    

In [5]:
# A.4. Kiểm tra dòng trùng lặp (theo ASSETID)
print("=" * 60)
print("A.4. KIỂM TRA DÒNG TRÙNG LẶP")
print("=" * 60)

# Trùng lặp hoàn toàn
full_duplicates = df.duplicated().sum()
print(f"Số dòng trùng lặp hoàn toàn: {full_duplicates}")

# Trùng lặp theo ASSETID
if 'ASSETID' in df.columns:
    assetid_duplicates = df['ASSETID'].duplicated().sum()
    print(f"Số ASSETID trùng lặp: {assetid_duplicates}")
    
    if assetid_duplicates > 0:
        print(f"\nCác ASSETID trùng lặp:")
        dup_assets = df[df['ASSETID'].duplicated(keep=False)].groupby('ASSETID').size()
        for asset, count in dup_assets.head(10).items():
            print(f"  - {asset}: {count} lần")

A.4. KIỂM TRA DÒNG TRÙNG LẶP
Số dòng trùng lặp hoàn toàn: 0
Số ASSETID trùng lặp: 0


In [6]:
# A.5. Kiểm tra các giá trị bất thường (khoảng trắng thừa)
print("=" * 60)
print("A.5. KIỂM TRA GIÁ TRỊ BẤT THƯỜNG")
print("=" * 60)

whitespace_issues = []
for col in df.columns:
    if df[col].dtype == 'object':
        # Kiểm tra khoảng trắng đầu/cuối
        mask = df[col].notna() & (df[col] != 'NULL')
        if mask.any():
            values_with_ws = df[mask][col].apply(
                lambda x: str(x) != str(x).strip() if pd.notna(x) else False
            ).sum()
            if values_with_ws > 0:
                whitespace_issues.append({'Cột': col, 'Số giá trị có khoảng trắng thừa': values_with_ws})

if whitespace_issues:
    ws_df = pd.DataFrame(whitespace_issues)
    print(ws_df.to_string(index=False))
else:
    print("Không phát hiện khoảng trắng thừa")

# Kiểm tra giá trị năm sản xuất bất thường
if 'DATEMANUFACTURE' in df.columns:
    print(f"\nKiểm tra DATEMANUFACTURE:")
    df['DATEMANUFACTURE_numeric'] = pd.to_numeric(df['DATEMANUFACTURE'], errors='coerce')
    invalid_years = df[
        (df['DATEMANUFACTURE_numeric'] < 1950) | 
        (df['DATEMANUFACTURE_numeric'] > datetime.now().year + 1)
    ]
    print(f"Số giá trị năm không hợp lệ (< 1950 hoặc > {datetime.now().year + 1}): {len(invalid_years)}")
    df.drop('DATEMANUFACTURE_numeric', axis=1, inplace=True)

A.5. KIỂM TRA GIÁ TRỊ BẤT THƯỜNG
      Cột  Số giá trị có khoảng trắng thừa
ASSETDESC                               12

Kiểm tra DATEMANUFACTURE:
Số giá trị năm không hợp lệ (< 1950 hoặc > 2027): 0


## B. Làm sạch dữ liệu

In [7]:
# Lưu thống kê trước khi làm sạch
stats_before = {
    'total_rows': len(df),
    'total_null_strings': (df == 'NULL').sum().sum(),
    'total_nan': df.isna().sum().sum(),
    'duplicate_rows': df.duplicated().sum(),
    'duplicate_assetid': df['ASSETID'].duplicated().sum() if 'ASSETID' in df.columns else 0
}
print("Thống kê TRƯỚC khi làm sạch:")
for k, v in stats_before.items():
    print(f"  {k}: {v:,}")

Thống kê TRƯỚC khi làm sạch:
  total_rows: 1,755
  total_null_strings: 0
  total_nan: 1,078
  duplicate_rows: 0
  duplicate_assetid: 0


In [8]:
# B.1. Xử lý NULL (chuyển 'NULL' string thành NaN)
print("=" * 60)
print("B.1. XỬ LÝ GIÁ TRỊ NULL")
print("=" * 60)

df_cleaned = df.copy()

# Chuyển 'NULL' string thành NaN
for col in df_cleaned.columns:
    df_cleaned[col] = df_cleaned[col].replace('NULL', np.nan)

null_converted = (df == 'NULL').sum().sum()
print(f"Đã chuyển {null_converted:,} giá trị 'NULL' string thành NaN")

B.1. XỬ LÝ GIÁ TRỊ NULL
Đã chuyển 0 giá trị 'NULL' string thành NaN


In [9]:
# B.2. Cắt khoảng trắng thừa ở mọi cột string
print("=" * 60)
print("B.2. CẮT KHOẢNG TRẮNG THỪA")
print("=" * 60)

trimmed_count = 0
for col in df_cleaned.columns:
    if df_cleaned[col].dtype == 'object':
        # Đếm số giá trị cần trim
        mask = df_cleaned[col].notna()
        before_trim = df_cleaned[mask][col].apply(lambda x: str(x))
        after_trim = before_trim.str.strip()
        trimmed_count += (before_trim != after_trim).sum()
        
        # Thực hiện trim
        df_cleaned[col] = df_cleaned[col].apply(
            lambda x: str(x).strip() if pd.notna(x) else x
        )

print(f"Đã cắt khoảng trắng thừa cho {trimmed_count:,} giá trị")

B.2. CẮT KHOẢNG TRẮNG THỪA
Đã cắt khoảng trắng thừa cho 12 giá trị


In [10]:
# B.3. Chuẩn hóa kiểu dữ liệu (DATEMANUFACTURE là numeric/int)
print("=" * 60)
print("B.3. CHUẨN HÓA KIỂU DỮ LIỆU")
print("=" * 60)

# Chuẩn hóa DATEMANUFACTURE
if 'DATEMANUFACTURE' in df_cleaned.columns:
    df_cleaned['DATEMANUFACTURE'] = pd.to_numeric(df_cleaned['DATEMANUFACTURE'], errors='coerce')
    # Chuyển thành Int64 (nullable integer)
    df_cleaned['DATEMANUFACTURE'] = df_cleaned['DATEMANUFACTURE'].astype('Int64')
    print(f"DATEMANUFACTURE: Đã chuyển sang kiểu Int64")
    print(f"  - Giá trị min: {df_cleaned['DATEMANUFACTURE'].min()}")
    print(f"  - Giá trị max: {df_cleaned['DATEMANUFACTURE'].max()}")
    print(f"  - Số giá trị NaN: {df_cleaned['DATEMANUFACTURE'].isna().sum()}")

B.3. CHUẨN HÓA KIỂU DỮ LIỆU
DATEMANUFACTURE: Đã chuyển sang kiểu Int64
  - Giá trị min: 1994
  - Giá trị max: 2025
  - Số giá trị NaN: 160


In [11]:
# B.3.1. DIEN DATEMANUFACTURE THEO NHOM (Group-based Imputation)
print("=" * 60)
print("B.3.1. DIEN DATEMANUFACTURE THEO NHOM")
print("=" * 60)

if 'DATEMANUFACTURE' in df_cleaned.columns and 'P_MANUFACTURERID' in df_cleaned.columns:
    nan_before = df_cleaned['DATEMANUFACTURE'].isna().sum()
    print(f"So gia tri NaN truoc khi dien: {nan_before}")
    
    # Buoc 1: Dien bang median theo nha san xuat
    df_cleaned['DATEMANUFACTURE'] = df_cleaned.groupby('P_MANUFACTURERID')['DATEMANUFACTURE'].transform(
        lambda x: x.fillna(x.median())
    )
    nan_after_step1 = df_cleaned['DATEMANUFACTURE'].isna().sum()
    print(f"Sau khi dien theo nha san xuat: {nan_after_step1} NaN con lai")
    
    # Buoc 2: Neu van con NaN, dien bang median chung
    if nan_after_step1 > 0:
        overall_median = df_cleaned['DATEMANUFACTURE'].median()
        df_cleaned['DATEMANUFACTURE'] = df_cleaned['DATEMANUFACTURE'].fillna(overall_median)
        nan_after_step2 = df_cleaned['DATEMANUFACTURE'].isna().sum()
        print(f"Sau khi dien bang median chung ({overall_median:.0f}): {nan_after_step2} NaN con lai")
    
    # Chuyen ve Int64
    df_cleaned['DATEMANUFACTURE'] = df_cleaned['DATEMANUFACTURE'].astype('Int64')
    
    print(f"\nKet qua:")
    print(f"  - Da dien {nan_before - df_cleaned['DATEMANUFACTURE'].isna().sum()} gia tri")
    print(f"  - Min: {df_cleaned['DATEMANUFACTURE'].min()}")
    print(f"  - Max: {df_cleaned['DATEMANUFACTURE'].max()}")
    print(f"  - Median: {df_cleaned['DATEMANUFACTURE'].median():.0f}")

B.3.1. DIEN DATEMANUFACTURE THEO NHOM
So gia tri NaN truoc khi dien: 160
Sau khi dien theo nha san xuat: 7 NaN con lai
Sau khi dien bang median chung (2015): 0 NaN con lai

Ket qua:
  - Da dien 160 gia tri
  - Min: 1994
  - Max: 2025
  - Median: 2015


In [12]:
# B.4. Bỏ dòng trùng lặp (ưu tiên giữ bản ghi đầy đủ hơn)
print("=" * 60)
print("B.4. XỬ LÝ DÒNG TRÙNG LẶP")
print("=" * 60)

if 'ASSETID' in df_cleaned.columns:
    # Tính số cột không null cho mỗi dòng
    df_cleaned['_non_null_count'] = df_cleaned.notna().sum(axis=1)
    
    # Sắp xếp theo ASSETID và số cột không null (giảm dần)
    df_cleaned = df_cleaned.sort_values(['ASSETID', '_non_null_count'], ascending=[True, False])
    
    # Giữ bản ghi đầu tiên (đầy đủ nhất) cho mỗi ASSETID
    rows_before = len(df_cleaned)
    df_cleaned = df_cleaned.drop_duplicates(subset=['ASSETID'], keep='first')
    rows_after = len(df_cleaned)
    
    # Xóa cột tạm
    df_cleaned = df_cleaned.drop('_non_null_count', axis=1)
    
    print(f"Số dòng trước khi xử lý: {rows_before:,}")
    print(f"Số dòng sau khi xử lý: {rows_after:,}")
    print(f"Số dòng đã loại bỏ: {rows_before - rows_after:,}")

B.4. XỬ LÝ DÒNG TRÙNG LẶP
Số dòng trước khi xử lý: 1,755
Số dòng sau khi xử lý: 1,755
Số dòng đã loại bỏ: 0


In [13]:
# B.5. Loại bỏ dòng có CATEGORYID không hợp lệ
print("=" * 60)
print("B.5. LOẠI BỎ DÒNG CÓ CATEGORYID KHÔNG HỢP LỆ")
print("=" * 60)

# Định nghĩa CATEGORYID hợp lệ (máy cắt)
VALID_CATEGORYID = '0110D00_MC'

if 'CATEGORYID' in df_cleaned.columns:
    # Đếm số dòng có CATEGORYID không hợp lệ
    invalid_category_mask = df_cleaned['CATEGORYID'] != VALID_CATEGORYID
    invalid_count = invalid_category_mask.sum()
    
    if invalid_count > 0:
        print(f"Số dòng có CATEGORYID không hợp lệ: {invalid_count}")
        print(f"\nCác CATEGORYID không hợp lệ:")
        invalid_categories = df_cleaned[invalid_category_mask]['CATEGORYID'].value_counts()
        for cat, count in invalid_categories.items():
            print(f"  - '{cat}': {count} dòng")
        
        # Hiển thị các dòng sẽ bị xóa
        print(f"\nVí dụ các dòng sẽ bị xóa:")
        print(df_cleaned[invalid_category_mask][['CATEGORYID', 'ASSETID', 'ASSETDESC']].head(5).to_string(index=False))
        
        # Xóa các dòng có CATEGORYID không hợp lệ
        rows_before = len(df_cleaned)
        df_cleaned = df_cleaned[~invalid_category_mask].copy()
        rows_after = len(df_cleaned)
        print(f"\n✓ Đã xóa {rows_before - rows_after} dòng có CATEGORYID không hợp lệ")
    else:
        print(f"✓ Tất cả các dòng đều có CATEGORYID hợp lệ: {VALID_CATEGORYID}")

B.5. LOẠI BỎ DÒNG CÓ CATEGORYID KHÔNG HỢP LỆ
Số dòng có CATEGORYID không hợp lệ: 1

Các CATEGORYID không hợp lệ:
  - '/1s': 1 dòng

Ví dụ các dòng sẽ bị xóa:
CATEGORYID            ASSETID           ASSETDESC
       /1s TBI_CT_MC_CT_DC_01 O-0,3sec-CO-3min-CO

✓ Đã xóa 1 dòng có CATEGORYID không hợp lệ


In [14]:
# B.6. Loại bỏ dòng thiếu quá nhiều dữ liệu (> 50% cột rỗng)
print("=" * 60)
print("B.6. LOẠI BỎ DÒNG THIẾU QUÁ NHIỀU DỮ LIỆU")
print("=" * 60)

# Các cột dữ liệu chính (không tính các cột _DESC vì thường là mô tả)
CORE_COLUMNS = [
    'CATEGORYID', 'ASSETID', 'ASSETDESC', 'P_MANUFACTURERID', 
    'DATEMANUFACTURE', 'NATIONALFACT', 'OWNER', 'LOAI', 
    'U_TT', 'KIEU_DAPHQ', 'I_DM', 'U_DM', 'KIEU_CD', 
    'TG_CATNM', 'PHA', 'KIEU_MC', 'KNCDNMDM', 'CT_DC'
]

# Lọc các cột tồn tại
existing_core_cols = [col for col in CORE_COLUMNS if col in df_cleaned.columns]
print(f"Số cột chính để kiểm tra: {len(existing_core_cols)}")

# Tính tỷ lệ cột rỗng cho mỗi dòng
df_cleaned['_missing_ratio'] = df_cleaned[existing_core_cols].isna().sum(axis=1) / len(existing_core_cols)

# Ngưỡng: loại bỏ dòng có > 50% cột rỗng
MISSING_THRESHOLD = 0.5

# Đếm số dòng vượt ngưỡng
high_missing_mask = df_cleaned['_missing_ratio'] > MISSING_THRESHOLD
high_missing_count = high_missing_mask.sum()

print(f"Ngưỡng loại bỏ: > {MISSING_THRESHOLD*100:.0f}% cột rỗng")
print(f"Số dòng vượt ngưỡng: {high_missing_count}")

if high_missing_count > 0:
    # Hiển thị phân bố tỷ lệ missing
    print(f"\nPhân bố tỷ lệ cột rỗng:")
    print(df_cleaned['_missing_ratio'].describe())
    
    # Hiển thị các dòng sẽ bị xóa
    print(f"\nVí dụ các dòng sẽ bị xóa (có nhiều cột rỗng):")
    cols_to_show = ['ASSETID', 'ASSETDESC', 'P_MANUFACTURERID', 'LOAI', '_missing_ratio']
    cols_to_show = [c for c in cols_to_show if c in df_cleaned.columns]
    print(df_cleaned[high_missing_mask][cols_to_show].head(5).to_string(index=False))
    
    # Xóa các dòng có quá nhiều cột rỗng
    rows_before = len(df_cleaned)
    df_cleaned = df_cleaned[~high_missing_mask].copy()
    rows_after = len(df_cleaned)
    print(f"\n✓ Đã xóa {rows_before - rows_after} dòng có > {MISSING_THRESHOLD*100:.0f}% cột rỗng")

# Xóa cột tạm
df_cleaned = df_cleaned.drop('_missing_ratio', axis=1)
print(f"\nSố dòng còn lại: {len(df_cleaned):,}")

B.6. LOẠI BỎ DÒNG THIẾU QUÁ NHIỀU DỮ LIỆU
Số cột chính để kiểm tra: 18
Ngưỡng loại bỏ: > 50% cột rỗng
Số dòng vượt ngưỡng: 39

Phân bố tỷ lệ cột rỗng:
count    1754.000000
mean        0.013715
std         0.091123
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         0.833333
Name: _missing_ratio, dtype: float64

Ví dụ các dòng sẽ bị xóa (có nhiều cột rỗng):
             ASSETID  ASSETDESC P_MANUFACTURERID LOAI  _missing_ratio
             MC.5327        NaN              NaN  NaN        0.833333
PC-0110D00-MC1065330  MC112_DMI        HSX.00311  NaN        0.611111
PC-0110D00-MC2926000  MC112_DNO        HSX.00183  NaN        0.611111
PC-0110D00-MC2959170  MC174_CJU        HSX.00311  NaN        0.611111
PC-0110D00-MC2985509 MC131_DRL2        HSX.00015  NaN        0.611111

✓ Đã xóa 39 dòng có > 50% cột rỗng

Số dòng còn lại: 1,715


In [15]:
# B.7. Kiểm tra và loại bỏ ASSETID không hợp lệ
print("=" * 60)
print("B.7. KIỂM TRA VÀ LOẠI BỎ ASSETID KHÔNG HỢP LỆ")
print("=" * 60)

import re

# Định nghĩa pattern hợp lệ cho ASSETID
# Các pattern phổ biến: MC.xxxx, PC-xxxx, PB-xxxx, PC14KK_T_MC_xxxx
VALID_ASSETID_PATTERNS = [
    r'^MC\.\d+$',                        # MC.5662
    r'^PC-0110D00-MC\d+$',              # PC-0110D00-MC1065330
    r'^PB-0110D00-MC\d+$',              # PB-0110D00-MC11485651
    r'^PC14KK_T_MC_\d+$',               # PC14KK_T_MC_1709
]

def is_valid_assetid(assetid):
    """Kiểm tra ASSETID có hợp lệ không"""
    if pd.isna(assetid):
        return False
    assetid_str = str(assetid).strip()
    for pattern in VALID_ASSETID_PATTERNS:
        if re.match(pattern, assetid_str):
            return True
    return False

if 'ASSETID' in df_cleaned.columns:
    # Kiểm tra ASSETID
    df_cleaned['_valid_assetid'] = df_cleaned['ASSETID'].apply(is_valid_assetid)
    invalid_assetid_mask = ~df_cleaned['_valid_assetid']
    invalid_count = invalid_assetid_mask.sum()
    
    print(f"Số dòng có ASSETID không hợp lệ: {invalid_count}")
    
    if invalid_count > 0:
        # Hiển thị các ASSETID không hợp lệ
        print(f"\nCác ASSETID không hợp lệ:")
        invalid_assetids = df_cleaned[invalid_assetid_mask]['ASSETID'].head(10).tolist()
        for assetid in invalid_assetids:
            print(f"  - '{assetid}'")
        
        # Xóa các dòng có ASSETID không hợp lệ
        rows_before = len(df_cleaned)
        df_cleaned = df_cleaned[~invalid_assetid_mask].copy()
        rows_after = len(df_cleaned)
        print(f"\n✓ Đã xóa {rows_before - rows_after} dòng có ASSETID không hợp lệ")
    else:
        print("✓ Tất cả ASSETID đều hợp lệ")
    
    # Xóa cột tạm
    if '_valid_assetid' in df_cleaned.columns:
        df_cleaned = df_cleaned.drop('_valid_assetid', axis=1)

print(f"\nSố dòng còn lại: {len(df_cleaned):,}")

B.7. KIỂM TRA VÀ LOẠI BỎ ASSETID KHÔNG HỢP LỆ
Số dòng có ASSETID không hợp lệ: 15

Các ASSETID không hợp lệ:
  - 'PB.1527365'
  - 'PB.1527380'
  - 'PB.1528573'
  - 'PB.1794298'
  - 'PB.1827994'
  - 'PB.1835992'
  - 'PB.2103087'
  - 'PB.2163703'
  - 'PB.2189874'
  - 'PB.2280377'

✓ Đã xóa 15 dòng có ASSETID không hợp lệ

Số dòng còn lại: 1,700


In [16]:
# B.8. Loại bỏ dòng có NATIONALFACT bị cấm
print("=" * 60)
print("B.8. LOẠI BỎ DÒNG CÓ NATIONALFACT BỊ CẤM")
print("=" * 60)

# Giá trị NATIONALFACT bị cấm
FORBIDDEN_NATIONALFACT = 'TB040.00023'

if 'NATIONALFACT' in df_cleaned.columns:
    forbidden_mask = df_cleaned['NATIONALFACT'] == FORBIDDEN_NATIONALFACT
    forbidden_count = forbidden_mask.sum()
    
    print(f"Số dòng có NATIONALFACT = {FORBIDDEN_NATIONALFACT}: {forbidden_count}")
    
    if forbidden_count > 0:
        # Hiển thị các dòng sẽ bị xóa
        print(f"\nCác dòng sẽ bị xóa:")
        cols_to_show = ['ASSETID', 'ASSETDESC', 'NATIONALFACT', 'FIELDDESC']
        cols_to_show = [c for c in cols_to_show if c in df_cleaned.columns]
        print(df_cleaned[forbidden_mask][cols_to_show].to_string(index=False))
        
        # Xóa các dòng vi phạm
        rows_before = len(df_cleaned)
        df_cleaned = df_cleaned[~forbidden_mask].copy()
        rows_after = len(df_cleaned)
        print(f"\n✓ Đã xóa {rows_before - rows_after} dòng có NATIONALFACT bị cấm")
    else:
        print("✓ Không có dòng nào vi phạm")

print(f"\nSố dòng còn lại: {len(df_cleaned):,}")

B.8. LOẠI BỎ DÒNG CÓ NATIONALFACT BỊ CẤM
Số dòng có NATIONALFACT = TB040.00023: 4

Các dòng sẽ bị xóa:
              ASSETID                       ASSETDESC NATIONALFACT FIELDDESC
              MC.5784      CT04THUDUCBAC-MC471-5.2015  TB040.00023  Việt Nam
PB-0110D00-MC14163629           MC 131 - TBA CHỢ LÁCH  TB040.00023  Việt Nam
PB-0110D00-MC14163640 MÁY CẮT 112 TRẠM 110KV CHỢ LÁCH  TB040.00023  Việt Nam
PB-0110D00-MC16460038           CT04TDB-MC171-12/2022  TB040.00023  Việt Nam

✓ Đã xóa 4 dòng có NATIONALFACT bị cấm

Số dòng còn lại: 1,696


In [17]:
# B.9. Tổng kết bước làm sạch
print("=" * 60)
print("B.9. TỔNG KẾT BƯỚC LÀM SẠCH")
print("=" * 60)

rows_removed = stats_before['total_rows'] - len(df_cleaned)
print(f"\nSố dòng ban đầu: {stats_before['total_rows']:,}")
print(f"Số dòng đã xóa: {rows_removed:,}")
print(f"Số dòng còn lại: {len(df_cleaned):,}")
print(f"Tỷ lệ dữ liệu giữ lại: {len(df_cleaned)/stats_before['total_rows']*100:.1f}%")

# Reset index sau khi xóa các dòng
df_cleaned = df_cleaned.reset_index(drop=True)
print("\n✓ Đã reset index của DataFrame")

B.9. TỔNG KẾT BƯỚC LÀM SẠCH

Số dòng ban đầu: 1,755
Số dòng đã xóa: 59
Số dòng còn lại: 1,696
Tỷ lệ dữ liệu giữ lại: 96.6%

✓ Đã reset index của DataFrame


## C. Áp dụng quy tắc chuẩn hóa (bắt buộc)

In [18]:
# Định nghĩa các quy tắc chuẩn hóa
NORMALIZATION_RULES = {
    'PHA': 'EVN.PHA_3P',                      # Pha
    'KIEU_MC': 'TBI_CT_MC_KIEU_MC_01',        # Kiểu máy cắt
    'KIEU_DAPHQ': 'TBI_TT_MC_KIEU_DAPHQ.00001',  # Kiểu đập hồ quang
    'KIEU_CD': 'TBI_CT_MC_CC_CD.00001',       # Kiểu cơ cấu đóng
    'U_TT': 'TBI_CT_MC_U_TT_02',              # Điện áp thao tác
}

# Giá trị cấm cho NATIONALFACT
FORBIDDEN_NATIONALFACT = 'TB040.00023'

print("=" * 60)
print("C. QUY TẮC CHUẨN HÓA")
print("=" * 60)
print("\nCác quy tắc chuẩn hóa bắt buộc:")
for col, value in NORMALIZATION_RULES.items():
    print(f"  - {col} = {value}")
print(f"  - NATIONALFACT không được là {FORBIDDEN_NATIONALFACT}")

C. QUY TẮC CHUẨN HÓA

Các quy tắc chuẩn hóa bắt buộc:
  - PHA = EVN.PHA_3P
  - KIEU_MC = TBI_CT_MC_KIEU_MC_01
  - KIEU_DAPHQ = TBI_TT_MC_KIEU_DAPHQ.00001
  - KIEU_CD = TBI_CT_MC_CC_CD.00001
  - U_TT = TBI_CT_MC_U_TT_02
  - NATIONALFACT không được là TB040.00023


In [19]:
# C.1-5. Áp dụng quy tắc chuẩn hóa
print("=" * 60)
print("C.1-5. ÁP DỤNG QUY TẮC CHUẨN HÓA")
print("=" * 60)

def apply_normalization_rules(df, rules):
    """Áp dụng các quy tắc chuẩn hóa cho DataFrame"""
    stats = {}
    
    for col, expected_value in rules.items():
        if col in df.columns:
            # Đếm số giá trị không đúng chuẩn
            mask_not_null = df[col].notna()
            mask_not_standard = mask_not_null & (df[col] != expected_value)
            non_standard_count = mask_not_standard.sum()
            
            # Lưu các giá trị không chuẩn để báo cáo
            non_standard_values = df[mask_not_standard][col].value_counts().head(5).to_dict()
            
            stats[col] = {
                'expected': expected_value,
                'non_standard_count': non_standard_count,
                'non_standard_values': non_standard_values
            }
            
            # Chuẩn hóa: giữ nguyên giá trị nếu đã có, hoặc để NaN nếu thiếu
            # Đánh dấu các giá trị không chuẩn để báo cáo
            print(f"\n{col}:")
            print(f"  - Giá trị chuẩn: {expected_value}")
            print(f"  - Số giá trị không chuẩn: {non_standard_count}")
            if non_standard_values:
                print(f"  - Top giá trị không chuẩn: {non_standard_values}")
        else:
            print(f"\n{col}: Không tìm thấy cột này trong DataFrame")
    
    return stats

normalization_stats = apply_normalization_rules(df_cleaned, NORMALIZATION_RULES)

C.1-5. ÁP DỤNG QUY TẮC CHUẨN HÓA

PHA:
  - Giá trị chuẩn: EVN.PHA_3P
  - Số giá trị không chuẩn: 0

KIEU_MC:
  - Giá trị chuẩn: TBI_CT_MC_KIEU_MC_01
  - Số giá trị không chuẩn: 6
  - Top giá trị không chuẩn: {'TBI_CT_MC_KIEU_MC_03': 3, 'TBI_CT_MC_KIEU_MC_02': 3}

KIEU_DAPHQ:
  - Giá trị chuẩn: TBI_TT_MC_KIEU_DAPHQ.00001
  - Số giá trị không chuẩn: 11
  - Top giá trị không chuẩn: {'TBI_TT_MC_KIEU_DAPHQ.10001': 11}

KIEU_CD:
  - Giá trị chuẩn: TBI_CT_MC_CC_CD.00001
  - Số giá trị không chuẩn: 273
  - Top giá trị không chuẩn: {'TBI_TT_MC_CC_CD.00001': 273}

U_TT:
  - Giá trị chuẩn: TBI_CT_MC_U_TT_02
  - Số giá trị không chuẩn: 1494
  - Top giá trị không chuẩn: {'TBI_CT_MC_U_TT_01': 1408, 'TBI_CT_MC_U_TT_04': 80, '110': 5, 'TBI_CT_MC_U_TT_03': 1}


In [20]:
# C.6. Xác nhận NATIONALFACT đã được làm sạch
print("=" * 60)
print("C.6. XÁC NHẬN NATIONALFACT ĐÃ ĐƯỢC LÀM SẠCH")
print("=" * 60)

if 'NATIONALFACT' in df_cleaned.columns:
    forbidden_count = (df_cleaned['NATIONALFACT'] == FORBIDDEN_NATIONALFACT).sum()
    print(f"Số bản ghi có NATIONALFACT = {FORBIDDEN_NATIONALFACT}: {forbidden_count}")
    
    if forbidden_count > 0:
        print(f"⚠️ Vẫn còn {forbidden_count} bản ghi vi phạm (cần kiểm tra lại bước B.8)")
    else:
        print("✓ Đã xóa tất cả bản ghi có NATIONALFACT bị cấm ở bước B.8")
    
    # Hiển thị phân bố NATIONALFACT
    print(f"\nPhân bố NATIONALFACT:")
    print(df_cleaned['NATIONALFACT'].value_counts().head(10))
else:
    print("Không tìm thấy cột NATIONALFACT")

C.6. XÁC NHẬN NATIONALFACT ĐÃ ĐƯỢC LÀM SẠCH
Số bản ghi có NATIONALFACT = TB040.00023: 0
✓ Đã xóa tất cả bản ghi có NATIONALFACT bị cấm ở bước B.8

Phân bố NATIONALFACT:
NATIONALFACT
TB040.00002    716
TB040.00006    690
TB040.00021    176
TB040.00016     40
TB040.00018     40
TB040.00141     21
TB040.00113      4
TB040.00008      3
TB040.00076      3
TB040.00001      2
Name: count, dtype: int64


In [21]:
# C.7. Tạo cột đánh dấu các giá trị không chuẩn (cho mục đích debug)
print("=" * 60)
print("C.7. TẠO CỘT ĐÁNH DẤU GIÁ TRỊ KHÔNG CHUẨN")
print("=" * 60)

print("Lưu ý: Các cột _FLAG được tạo để debug, không xuất ra file cleaned chính.")
print("Giá trị không chuẩn được GIỮ NGUYÊN (không sửa đổi) trong dữ liệu.")
print()

# Đánh dấu các cột có giá trị không chuẩn
total_nonstandard = 0
for col, expected_value in NORMALIZATION_RULES.items():
    if col in df_cleaned.columns:
        flag_col = f'_FLAG_{col}_NONSTANDARD'
        df_cleaned[flag_col] = (
            df_cleaned[col].notna() & (df_cleaned[col] != expected_value)
        )
        count = df_cleaned[flag_col].sum()
        total_nonstandard += count
        print(f"  {col}: {count} giá trị không chuẩn (expected: {expected_value})")

print(f"\nTổng số giá trị không chuẩn: {total_nonstandard}")

C.7. TẠO CỘT ĐÁNH DẤU GIÁ TRỊ KHÔNG CHUẨN
Lưu ý: Các cột _FLAG được tạo để debug, không xuất ra file cleaned chính.
Giá trị không chuẩn được GIỮ NGUYÊN (không sửa đổi) trong dữ liệu.

  PHA: 0 giá trị không chuẩn (expected: EVN.PHA_3P)
  KIEU_MC: 6 giá trị không chuẩn (expected: TBI_CT_MC_KIEU_MC_01)
  KIEU_DAPHQ: 11 giá trị không chuẩn (expected: TBI_TT_MC_KIEU_DAPHQ.00001)
  KIEU_CD: 273 giá trị không chuẩn (expected: TBI_CT_MC_CC_CD.00001)
  U_TT: 1494 giá trị không chuẩn (expected: TBI_CT_MC_U_TT_02)

Tổng số giá trị không chuẩn: 1784


## D. Bổ sung dữ liệu thiếu

In [22]:
# D.1. Điền giá trị phổ biến nhất cho các cột bắt buộc (nếu hợp lệ)
print("=" * 60)
print("D.1. ĐIỀN GIÁ TRỊ PHỔ BIẾN NHẤT")
print("=" * 60)

# Các cột có thể điền giá trị mặc định
FILLABLE_COLUMNS = ['OWNER', 'CATEGORYID']

for col in FILLABLE_COLUMNS:
    if col in df_cleaned.columns:
        null_count = df_cleaned[col].isna().sum()
        if null_count > 0:
            most_common = df_cleaned[col].mode()
            if len(most_common) > 0:
                most_common_value = most_common[0]
                df_cleaned[col] = df_cleaned[col].fillna(most_common_value)
                print(f"{col}: Đã điền {null_count} giá trị NaN bằng '{most_common_value}'")
        else:
            print(f"{col}: Không có giá trị NaN")

D.1. ĐIỀN GIÁ TRỊ PHỔ BIẾN NHẤT
OWNER: Không có giá trị NaN
CATEGORYID: Không có giá trị NaN


In [23]:
# D.2. Với cột kỹ thuật quan trọng → giữ NULL và gắn cờ "missing"
print("=" * 60)
print("D.2. ĐÁNH DẤU CỘT KỸ THUẬT THIẾU DỮ LIỆU")
print("=" * 60)

# Các cột kỹ thuật quan trọng (không nên điền giá trị mặc định)
TECHNICAL_COLUMNS = [
    'LOAI', 'U_TT', 'KIEU_DAPHQ', 'I_DM', 'U_DM', 
    'KIEU_CD', 'TG_CATNM', 'PHA', 'KIEU_MC', 'KNCDNMDM', 'CT_DC'
]

for col in TECHNICAL_COLUMNS:
    if col in df_cleaned.columns:
        null_count = df_cleaned[col].isna().sum()
        if null_count > 0:
            flag_col = f'_FLAG_{col}_MISSING'
            df_cleaned[flag_col] = df_cleaned[col].isna()
            print(f"{col}: {null_count} giá trị thiếu → đã tạo cột {flag_col}")

D.2. ĐÁNH DẤU CỘT KỸ THUẬT THIẾU DỮ LIỆU


## E. Xuất dữ liệu sạch

In [24]:
# E.1. Thống kê trước/sau làm sạch
print("=" * 60)
print("E.1. THỐNG KÊ TRƯỚC/SAU LÀM SẠCH")
print("=" * 60)

# Lọc bỏ các cột _FLAG trước khi thống kê
data_cols = [col for col in df_cleaned.columns if not col.startswith('_FLAG')]
df_data = df_cleaned[data_cols]

stats_after = {
    'total_rows': len(df_data),
    'total_null_strings': (df_data == 'NULL').sum().sum(),
    'total_nan': df_data.isna().sum().sum(),
    'duplicate_rows': df_data.duplicated().sum(),
    'duplicate_assetid': df_data['ASSETID'].duplicated().sum() if 'ASSETID' in df_data.columns else 0
}

print("\nSo sánh TRƯỚC và SAU khi làm sạch:")
print(f"{'Chỉ số':<25} {'Trước':>12} {'Sau':>12} {'Thay đổi':>12}")
print("-" * 55)
for key in stats_before.keys():
    before_val = stats_before[key]
    after_val = stats_after[key]
    change = after_val - before_val
    print(f"{key:<25} {before_val:>12,} {after_val:>12,} {change:>+12,}")

E.1. THỐNG KÊ TRƯỚC/SAU LÀM SẠCH

So sánh TRƯỚC và SAU khi làm sạch:
Chỉ số                           Trước          Sau     Thay đổi
-------------------------------------------------------
total_rows                       1,755        1,696          -59
total_null_strings                   0            0           +0
total_nan                        1,078           25       -1,053
duplicate_rows                       0            0           +0
duplicate_assetid                    0            0           +0


In [25]:
# E.2. Xuất file CSV đã làm sạch
print("=" * 60)
print("E.2. XUẤT FILE CSV")
print("=" * 60)

# Xuất file không có cột _FLAG (dữ liệu sạch)
df_export = df_cleaned[data_cols].copy()
df_export.to_csv(OUTPUT_FILE, sep=';', index=False, encoding='utf-8')
print(f"Đã xuất file: {OUTPUT_FILE}")
print(f"Số dòng: {len(df_export):,}")
print(f"Số cột: {len(df_export.columns)}")

# Xuất file với cột _FLAG (để debug)
FLAG_FILE = OUTPUT_FILE.replace('_cleaned.csv', '_cleaned_with_flags.csv')
df_cleaned.to_csv(FLAG_FILE, sep=';', index=False, encoding='utf-8')
print(f"\nĐã xuất file với flags: {FLAG_FILE}")

E.2. XUẤT FILE CSV


Đã xuất file: /home/aispcit/Documents/QuangLV/PMIS v 13/data/devicesPMISMayCat_cleaned.csv
Số dòng: 1,696
Số cột: 32

Đã xuất file với flags: /home/aispcit/Documents/QuangLV/PMIS v 13/data/devicesPMISMayCat_cleaned_with_flags.csv


In [26]:
# E.3. Ghi log thống kê
print("=" * 60)
print("E.3. GHI LOG THỐNG KÊ")
print("=" * 60)

log_file = os.path.join(LOG_DIR, f'preprocessing_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt')

with open(log_file, 'w', encoding='utf-8') as f:
    f.write("=" * 60 + "\n")
    f.write("LOG TIỀN XỬ LÝ DỮ LIỆU PMIS\n")
    f.write(f"Thời gian: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("=" * 60 + "\n\n")
    
    f.write("THỐNG KÊ TRƯỚC KHI LÀM SẠCH:\n")
    for k, v in stats_before.items():
        f.write(f"  {k}: {v:,}\n")
    
    f.write("\nTHỐNG KÊ SAU KHI LÀM SẠCH:\n")
    for k, v in stats_after.items():
        f.write(f"  {k}: {v:,}\n")
    
    f.write("\nQUY TẮC CHUẨN HÓA ĐÃ ÁP DỤNG:\n")
    for col, value in NORMALIZATION_RULES.items():
        f.write(f"  {col} = {value}\n")
    f.write(f"  NATIONALFACT != {FORBIDDEN_NATIONALFACT}\n")
    
    f.write("\nFILE XUẤT:\n")
    f.write(f"  - {OUTPUT_FILE}\n")
    f.write(f"  - {FLAG_FILE}\n")

print(f"Đã ghi log: {log_file}")

E.3. GHI LOG THỐNG KÊ
Đã ghi log: /home/aispcit/Documents/QuangLV/PMIS v 13/logs/preprocessing_log_20260202_122149.txt


In [27]:
# Tổng kết
print("\n" + "=" * 60)
print("✅ HOÀN TẤT TIỀN XỬ LÝ DỮ LIỆU")
print("=" * 60)
print(f"\nFile đầu vào: {INPUT_FILE}")
print(f"File đầu ra: {OUTPUT_FILE}")

print(f"\n📊 THỐNG KÊ:")
print(f"  - Số dòng ban đầu: {stats_before['total_rows']:,}")
print(f"  - Số dòng sau làm sạch: {len(df_export):,}")
print(f"  - Số dòng đã xóa: {stats_before['total_rows'] - len(df_export):,}")

print(f"\n✓ Các bước đã thực hiện:")
print("  A. Đọc & phân tích dữ liệu")
print("  B. Làm sạch dữ liệu:")
print("     B.1. Xử lý NULL string")
print("     B.2. Cắt khoảng trắng thừa")
print("     B.3. Chuẩn hóa kiểu dữ liệu")
print("     B.4. Xử lý dòng trùng lặp")
print("     B.5. Loại bỏ CATEGORYID không hợp lệ")
print("     B.6. Loại bỏ dòng thiếu > 50% dữ liệu")
print("     B.7. Loại bỏ ASSETID không hợp lệ")
print("     B.8. Loại bỏ NATIONALFACT bị cấm")
print("  C. Kiểm tra quy tắc chuẩn hóa")
print("  D. Đánh dấu dữ liệu thiếu")
print("  E. Xuất dữ liệu sạch")


✅ HOÀN TẤT TIỀN XỬ LÝ DỮ LIỆU

File đầu vào: /home/aispcit/Documents/QuangLV/PMIS v 13/data/devicesPMISMayCat.csv
File đầu ra: /home/aispcit/Documents/QuangLV/PMIS v 13/data/devicesPMISMayCat_cleaned.csv

📊 THỐNG KÊ:
  - Số dòng ban đầu: 1,755
  - Số dòng sau làm sạch: 1,696
  - Số dòng đã xóa: 59

✓ Các bước đã thực hiện:
  A. Đọc & phân tích dữ liệu
  B. Làm sạch dữ liệu:
     B.1. Xử lý NULL string
     B.2. Cắt khoảng trắng thừa
     B.3. Chuẩn hóa kiểu dữ liệu
     B.4. Xử lý dòng trùng lặp
     B.5. Loại bỏ CATEGORYID không hợp lệ
     B.6. Loại bỏ dòng thiếu > 50% dữ liệu
     B.7. Loại bỏ ASSETID không hợp lệ
     B.8. Loại bỏ NATIONALFACT bị cấm
  C. Kiểm tra quy tắc chuẩn hóa
  D. Đánh dấu dữ liệu thiếu
  E. Xuất dữ liệu sạch
